In [1]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor


In [13]:
LEARNING_RATE = 1e-3
BATCH_SIZE = 64
EPOCHS = 10


In [3]:
# Download training and test data

training_data = datasets.MNIST(
    root="data", train=True, download=True, transform=ToTensor()
)

test_data = datasets.MNIST(
    root="data", train=False, download=True, transform=ToTensor()
)


100.0%


Extracting data\MNIST\raw\train-images-idx3-ubyte.gz to data\MNIST\raw



100.0%


Extracting data\MNIST\raw\train-labels-idx1-ubyte.gz to data\MNIST\raw



100.0%


Extracting data\MNIST\raw\t10k-images-idx3-ubyte.gz to data\MNIST\raw



100.0%

Extracting data\MNIST\raw\t10k-labels-idx1-ubyte.gz to data\MNIST\raw



In [14]:
train_dataloader = DataLoader(training_data, batch_size=BATCH_SIZE)
test_dataloader = DataLoader(test_data, batch_size=BATCH_SIZE)


In [5]:
# Define neural network


class NeuralNetwork(nn.Module):
    def __init__(self):
        super(NeuralNetwork, self).__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28 * 28, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10),
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits


In [7]:
# Train and test loops


def train_loop(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    for batch, (X, y) in enumerate(dataloader):
        # Compute prediction and loss
        pred = model(X)
        loss = loss_fn(pred, y)

        # Backpropagation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if batch % 100 == 0:
            loss, current = loss.item(), (batch + 1) * len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")


def test_loop(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    test_loss, correct = 0, 0

    with torch.no_grad():
        for X, y in dataloader:
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()

    test_loss /= num_batches
    correct /= size
    print(
        f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n"
    )


In [8]:
model = NeuralNetwork()


In [9]:
loss_fn = nn.CrossEntropyLoss()


In [15]:
optimizer = torch.optim.SGD(model.parameters(), lr=LEARNING_RATE)


In [16]:
for t in range(EPOCHS):
    print(f"Epoch {t+1}\n-------------------------------")
    train_loop(train_dataloader, model, loss_fn, optimizer)
    test_loop(test_dataloader, model, loss_fn)
print("Done!")


Epoch 1
-------------------------------
loss: 2.301961  [   64/60000]
loss: 2.290491  [ 6464/60000]
loss: 2.294580  [12864/60000]
loss: 2.283357  [19264/60000]
loss: 2.287991  [25664/60000]
loss: 2.281233  [32064/60000]
loss: 2.263251  [38464/60000]
loss: 2.275008  [44864/60000]
loss: 2.262614  [51264/60000]
loss: 2.251699  [57664/60000]
Test Error: 
 Accuracy: 47.2%, Avg loss: 2.253463 

Epoch 2
-------------------------------
loss: 2.255571  [   64/60000]
loss: 2.240594  [ 6464/60000]
loss: 2.253427  [12864/60000]
loss: 2.220958  [19264/60000]
loss: 2.236372  [25664/60000]
loss: 2.227659  [32064/60000]
loss: 2.203604  [38464/60000]
loss: 2.227922  [44864/60000]
loss: 2.197822  [51264/60000]
loss: 2.184868  [57664/60000]
Test Error: 
 Accuracy: 60.9%, Avg loss: 2.185145 

Epoch 3
-------------------------------
loss: 2.186968  [   64/60000]
loss: 2.164454  [ 6464/60000]
loss: 2.191865  [12864/60000]
loss: 2.125302  [19264/60000]
loss: 2.155245  [25664/60000]
loss: 2.140745  [32064/600

In [17]:
torch.save(model, "model.pth")
